In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

In [11]:
# Read the data
PATH = '/content/drive/MyDrive/Dataset/'
df = pd.read_csv(PATH + 'csv_preprocessed.csv')
print(df.shape)

(1215675, 63)


In [12]:
# Define features and target
X = df.drop(columns=['Attack Type'])
y = df['Attack Type']

In [13]:
# Train Test split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Attack Type']), df['Attack Type'],
                                                    stratify=df['Attack Type'],
                                                    test_size=0.15,
                                                    random_state=42)

In [14]:
X_train = pd.DataFrame(X_train, columns=df.drop(columns=['Attack Type']).columns.to_list())
X_test = pd.DataFrame(X_test, columns=df.drop(columns=['Attack Type']).columns.to_list())
y_train = pd.DataFrame(y_train, columns=['Attack Type'])
y_test = pd.DataFrame(y_test, columns=['Attack Type'])

In [15]:
print("Training dataset size:", X_train.shape)
print("Testing dataset size:", X_test.shape)
print("Training target size:", y_train.shape)
print("Testing target size:", y_test.shape)

Training dataset size: (1033323, 62)
Testing dataset size: (182352, 62)
Training target size: (1033323, 1)
Testing target size: (182352, 1)


In [16]:
# Perform mutual information feature selection on training set
mutual_info_arr = mutual_info_classif(X_train, y_train)
top_features = X_train.columns[np.argsort(mutual_info_arr)[::-1][:15]]
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
# Perform PCA on selected features
pca = PCA(n_components=15)
X_train_pca = pca.fit_transform(X_train_selected)
X_test_pca = pca.transform(X_test_selected)

In [18]:
# Define the specific hyperparameters
best_params = {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}

In [19]:
# Initialize Random Forest Classifier with specific hyperparameters
rfc = RandomForestClassifier(**best_params, random_state=42)

In [20]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
# Lists to store evaluation metrics across folds
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

In [22]:
# Perform k-fold cross-validation
for train_index, test_index in skf.split(X_train_pca, y_train):
    X_train_fold, X_val_fold = X_train_pca[train_index], X_train_pca[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

In [23]:
    # Fit the model on the training fold
    rfc.fit(X_train_fold, y_train_fold)

<ipython-input-23-5baf39697865>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train_fold, y_train_fold)


RandomForestClassifier(bootstrap=False, criterion='entropy', min_samples_leaf=2,
                       n_estimators=50, random_state=42)

In [24]:
    # Predict on the validation fold
    y_val_pred = rfc.predict(X_val_fold)

In [25]:
    # Evaluate the model on the validation fold
    accuracy_fold = accuracy_score(y_val_fold, y_val_pred)
    precision_fold = precision_score(y_val_fold, y_val_pred, average='weighted')
    recall_fold = recall_score(y_val_fold, y_val_pred, average='weighted')
    f1_fold = f1_score(y_val_fold, y_val_pred, average='weighted')

In [26]:
    # Append scores to lists
    accuracy_scores.append(accuracy_fold)
    precision_scores.append(precision_fold)
    recall_scores.append(recall_fold)
    f1_scores.append(f1_fold)

In [27]:
# Average the evaluation metrics across folds
average_accuracy = np.mean(accuracy_scores)
average_precision = np.mean(precision_scores)
average_recall = np.mean(recall_scores)
average_f1 = np.mean(f1_scores)

In [28]:
# Predict on the X_test dataset
y_test_pred = rfc.predict(X_test_pca)

In [29]:
# Evaluate the model on the X_test dataset
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred, average='weighted')
recall_test = recall_score(y_test, y_test_pred, average='weighted')
f1_test = f1_score(y_test, y_test_pred, average='weighted')

In [30]:
# Print the average scores and accuracy on test dataset
print(f'Average Accuracy (cross-validation): {average_accuracy}')
print(f'Average Precision (cross-validation): {average_precision}')
print(f'Average Recall (cross-validation): {average_recall}')
print(f'Average F1 Score (cross-validation): {average_f1}')
print('\n')
print(f'Accuracy (test dataset): {accuracy_test}')
print(f'Precision (test dataset): {precision_test}')
print(f'Recall (test dataset): {recall_test}')
print(f'F1 Score (test dataset): {f1_test}')

Average Accuracy (cross-validation): 0.9966806023303526
Average Precision (cross-validation): 0.9966774928316217
Average Recall (cross-validation): 0.9966806023303526
Average F1 Score (cross-validation): 0.9966770696922933


Accuracy (test dataset): 0.996397078178468
Precision (test dataset): 0.9963940015317299
Recall (test dataset): 0.996397078178468
F1 Score (test dataset): 0.9963914668026004
